# Caffe的Python接口学习(4)：mnist实例——手写数字识别

深度学习的第一个实例一般都是mnist，只要这个例子学通了，其他的就是举一反三的事了。由于篇幅的原因，本文不具体介绍配置文件里面每个参数的具体涵义，如果想弄明白，请参考：

[数据层及参数](http://www.cnblogs.com/denny402/p/5070928.html)

[视觉层及参数](http://www.cnblogs.com/denny402/p/5071126.html)

[solver配置文件及参数](http://www.cnblogs.com/denny402/p/5074049.html)

## 一、数据准备

官网提供的mnist数据并不是图片，但我们以后做的实际项目可能是图片。因此有些人并不知道该怎么办。在此我将mnist数据进行了转化，变成了一张张的图片，我们练习就从图片开始。[点击下载mnist图片数据](https://pan.baidu.com/s/1pLMV4Kz)。

数据分成了训练集（60000张共10类）和测试集（共10000张10类），每个类别放在一个单独的文件夹里。并且将所有的图片，都生成了txt列表清单（train.txt和test.txt）。大家下载下来后，直接解压到当前用户目录下就可以了。由于是在Windows下压缩的，因此是winrar文件。如果大家要在Linux下解压所，需要安装rar的Linux版本，也是十分简单：

``` shell
# Ubuntu
sudo apt-get install rar
# CentOS
yum install rar
```

## 二、导入Caffe库，并设定文件路径

代码如下：

``` python
# -*- coding: utf-8 -*-

import caffe
from caffe import layers as L, params as P, proto, to_proto

# 设定文件的保存路径
root = '/root/Repository/caffe/examples/mine/' # 父目录
train_list = root + 'mnist/train/train.txt' # 训练图片列表
test_list = root + 'mnist/test/test.txt' # 测试图片列表
train_proto = root + 'mnist/train.prototxt' # 训练配置文件
test_proto = root + 'mnist/test.prototxt' # 测试配置文件
solver_proto = root + 'mnist/solver.prototxt' # 参数文件
```

其中train.txt和test.txt文件已经有了，其他三个文件，我们需要自己编写。

此处注意：一般caffe程序都是先将图片转换成LMDB文件，但这样做有点麻烦，因此我就不转换了，我直接用原始图片进行操作，所不同的是直接用图片操作，均值很难计算，因此可以不减均值。

## 三、生成配置文件

配置文件实际上就是一些txt文档，只是后缀名是prototxt，我们可以直接到编辑器里编写，也可以用代码生成。此处，我用Python来生成。

``` python
# 编写一个函数，生成配置文件prototxt
def lenet(img_list, batch_size, include_acc=False):
    # 数据输入层，以ImageData格式输入
    data, label = L.ImageData(source=img_list, batch_size=batch_size, ntop=2, root_folder=root,
                              transform_param=dict(scale=0.00390625))
    # 卷积层
    conv1 = L.Convolution(data, kernel_size=5, stride=1, num_output=20, pad=0, weight_filter=dict(type='xavier'))
    # 池化层
    pool1 = L.Pooling(conv1, pool=P.Pooling.MAX, kernel_size=2, stride=2)
    # 卷积层
    conv2 = L.Convolution(pool1, kernel_size=5, stride=1, num_output=50, pad=0, weight_filter=dict(type='xavier'))
    # 池化层
    pool2 = L.Pooling(conv2, pool=P.Pooling.MAX, kernel_size=2, stride=2)
    # 全连接层
    fc3 = L.InnerProduct(pool2, num_output=500, weight_filter=dict(type='xavier'))
    # 激活函数层
    relu3 = L.ReLU(fc3, in_place=True)
    # 全连接层
    fc4 = L.InnerProduct(relu3, num_output=10, weight_filter=dict(type='xavier'))
    # softmax层
    loss = L.SoftmaxWithLoss(fc4, label)
    
    # test阶段需要有accuracy层
    if include_acc:
        acc = L.Accuracy(fc4, label)
        return to_proto(loss, acc)
    else:
        return to_proto(loss)
    
def write_net():
    # 写入train.prototxt
    with open(train_proto, 'w') as f:
        f.write(str(lenet(train_list, batch_size=64)))
        
    # 写入test.prototxt
    with open(test_proto, 'w') as f:
        f.write(str(lenet(test_list, batch_size=100, include_acc=True)))
```

配置文件里面存放的，就是我们说的network。我这里生成的network，可能和原始的LeNet不太一样，但影响不大。

## 四、生成参数文件

同样，可以在编辑器里面直接书写，也可以用代码生成。

``` python
# 编写一个函数，生成参数文件
def get_solver(solver_file, train_net, test_net):
    s = proto.caffe_pb2.SolverParameter()
    s.train_net = train_net
    s.test_net.append(test_net)
    s.test_interval = 938 # 60000/64，测试间隔参数：训练完一次所有图片，进行一次测试
    s.test_iter.append(100) # 10000/100，测试迭代次数，需要迭代100次，才完成一次所有数据的测试
    s.max_iter = 9380 # 10 epochs，938*10，最大训练次数
    s.base_lr = 0.01 # 基础学习率
    s.momentum = 0.9 # 动量
    s.weight_decay = 5e-4 # 权值衰减项
    s.lr_policy = 'step' # 学习率变化规则
    s.stepsize = 3000 # 学习率变化频率
    s.gamma = 0.1 # 学习率变化指数
    s.display = 20 # 屏幕显示间隔
    s.snapshot = 9380 # 保存caffemodel的间隔
    s.snapshot_prefix = root + 'mnist/lenet' # caffemodel前缀
    s.type = 'SGD' # 优化算法
    s.solver_mode = proto.caffe_pb2.SolverParameter.CPU # CPU
    # 写入solver.prototxt
    with open(solver_file, 'w') as f:
        f.write(str(s))
```

## 五、开始训练模型

训练过程中，也在不停的测试。

``` python
# 开始训练
def training(solver_proto):
    # caffe.set_device(0)
    # caffe.set_mode_gpu()
    solver = caffe.SGDSolver(solver_proto)
    solver.solve()
```

最后，调用以上的函数就可以了。

``` python
if __name__ == '__main__':
    write_net()
    get_solver(solver_proto, train_proto, test_proto)
    training(solver_proto)
```

## 六、完整的Python文件

mnist.py：
``` python
# -*- coding: utf-8 -*-

import caffe
from caffe import layers as L, params as P, proto, to_proto

# 设定文件的保存路径
root = '/root/Repository/caffe/examples/mine/' # 父目录
train_list = root + 'mnist/train/train.txt' # 训练图片列表
test_list = root + 'mnist/test/test.txt' # 测试图片列表
train_proto = root + 'mnist/train.prototxt' # 训练配置文件
test_proto = root + 'mnist/test.prototxt' # 测试配置文件
solver_proto = root + 'mnist/solver.prototxt' # 参数文件

# 编写一个函数，生成配置文件prototxt
def lenet(img_list, batch_size, include_acc=False):
    # 数据输入层，以ImageData格式输入
    data, label = L.ImageData(source=img_list, batch_size=batch_size, ntop=2, root_folder=root,
                              transform_param=dict(scale=0.00390625))
    # 卷积层
    conv1 = L.Convolution(data, kernel_size=5, stride=1, num_output=20, pad=0, weight_filter=dict(type='xavier'))
    # 池化层
    pool1 = L.Pooling(conv1, pool=P.Pooling.MAX, kernel_size=2, stride=2)
    # 卷积层
    conv2 = L.Convolution(pool1, kernel_size=5, stride=1, num_output=50, pad=0, weight_filter=dict(type='xavier'))
    # 池化层
    pool2 = L.Pooling(conv2, pool=P.Pooling.MAX, kernel_size=2, stride=2)
    # 全连接层
    fc3 = L.InnerProduct(pool2, num_output=500, weight_filter=dict(type='xavier'))
    # 激活函数层
    relu3 = L.ReLU(fc3, in_place=True)
    # 全连接层
    fc4 = L.InnerProduct(relu3, num_output=10, weight_filter=dict(type='xavier'))
    # softmax层
    loss = L.SoftmaxWithLoss(fc4, label)
    
    # test阶段需要有accuracy层
    if include_acc:
        acc = L.Accuracy(fc4, label)
        return to_proto(loss, acc)
    else:
        return to_proto(loss)
    
def write_net():
    # 写入train.prototxt
    with open(train_proto, 'w') as f:
        f.write(str(lenet(train_list, batch_size=64)))
        
    # 写入test.prototxt
    with open(test_proto, 'w') as f:
        f.write(str(lenet(test_list, batch_size=100, include_acc=True)))

# 编写一个函数，生成参数文件
def get_solver(solver_file, train_net, test_net):
    s = proto.caffe_pb2.SolverParameter()
    s.train_net = train_net
    s.test_net.append(test_net)
    s.test_interval = 938 # 60000/64，测试间隔参数：训练完一次所有图片，进行一次测试
    s.test_iter.append(100) # 10000/100，测试迭代次数，需要迭代100次，才完成一次所有数据的测试
    s.max_iter = 9380 # 10 epochs，938*10，最大训练次数
    s.base_lr = 0.01 # 基础学习率
    s.momentum = 0.9 # 动量
    s.weight_decay = 5e-4 # 权值衰减项
    s.lr_policy = 'step' # 学习率变化规则
    s.stepsize = 3000 # 学习率变化频率
    s.gamma = 0.1 # 学习率变化指数
    s.display = 20 # 屏幕显示间隔
    s.snapshot = 9380 # 保存caffemodel的间隔
    s.snapshot_prefix = root + 'mnist/lenet' # caffemodel前缀
    s.type = 'SGD' # 优化算法
    s.solver_mode = proto.caffe_pb2.SolverParameter.CPU # CPU
    # 写入solver.prototxt
    with open(solver_file, 'w') as f:
        f.write(str(s))

# 开始训练
def training(solver_proto):
    # caffe.set_device(0)
    # caffe.set_mode_gpu()
    solver = caffe.SGDSolver(solver_proto)
    solver.solve()

if __name__ == '__main__':
    write_net()
    get_solver(solver_proto, train_proto, test_proto)
    training(solver_proto)
```

将此文件放在root指定的目录下，执行该代码：

``` shell
sudo python examples/mine/mnist.py
```

在训练过程中，会保存一些caffemodel。多久保存一次，保存多少次，都可以在solver参数文件里进行设置。

这里设置为训练10 epochs，9380次，测试精度可以达到99%。